In [ ]:
from __future__ import print_function
import os
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, fixed
from IPython.display import Image
import matplotlib.pyplot as plt
from labellines import labelLine, labelLines
from cycler import cycler


import sys
import time
import numpy as np

plt.style.use(['~/prash.mplstyle'])
# plt.rcParams['figure.figsize'] = (15, 5)
# plt.rcParams['lines.linewidth'] = 2
# plt.rcParams['lines.markersize'] = 15
# plt.rcParams['font.size']=16
# plt.rcParams['grid.linestyle']='--'
# plt.rcParams['grid.linewidth']= 0.1
# plt.rcParams['grid.alpha']= 0.5

In [ ]:
def ThermalEff(Comp_eff, Turb_eff, T4qT2, PR, gamma = 1.4):
    k = (gamma - 1)/gamma
    A = PR**k
    Eff_th = (Turb_eff*T4qT2*(1-1/A) - (A-1)/Comp_eff)/(T4qT2 - 1 - (A-1)/Comp_eff)
    
    return Eff_th
    
def Work_per_flow(Comp_eff, Turb_eff, T4qT2, PR, gamma = 1.4, turb_comp_works = False):
    """
    Calculates the Wnet/(mair*cp*Tt2)
    """
    k = (gamma - 1)/gamma
    A = PR**k
    
    AvailableWork = Turb_eff*T4qT2*(1-1/A)
    CompWork = (A-1)/Comp_eff
    Tt3qTt2 = (A-1)/Comp_eff + 1
    
    Wqflow = AvailableWork - CompWork
    
    if turb_comp_works:
        return Wqflow, AvailableWork, CompWork, Tt3qTt2
    else:
        return Wqflow
    
def Work_per_flow_w_PT(Comp_eff, Turb_eff, PowerTurb_eff, T4qT2, PR, gamma = 1.4, details = False):
    """
    Calculates the Wnet/(mair*cp*Tt2) with a power turbine
    """
    
    k = (gamma - 1)/gamma
    A = PR**k
    
    CompWqflow = (A-1)/Comp_eff # W_compr/(m_air*cp*Tt2) = (PR**(k) - 1)/eta  
    T5qT4      = 1 + CompWqflow/T4qT2
    T5qT4_is   = 1 + (T5qT4 - 1)/Turb_eff 
    P5qP4      = T5qT4_is**(1/k)
    P4qP5      = 1/P5qP4
    P6qP5      = P4qP5/PR
    
    T6qT5      = 1 + PowerTurb_eff*(P6qP5**k-1)
    
    Wqflow = T4qT2*(1 - T6qT5*T5qT4) - CompWqflow
#     Wqflow = T4qT2*(1-
#                     (1+PowerTurb_eff*
#                      (1-1/A*(1/(1-(A-1)/(Turb_eff*Comp_eff*T4qT2))))*(1-(A-1)/(T4qT2*Comp_eff))
#                     )
#                     -CompWqflow)

    
    if details:
        return Wqflow, T5qT4
    else:
        return Wqflow


In [ ]:
ThermalEff(0.9, .9,6.26 ,27.1, gamma=1.4)

In [ ]:
PR = np.linspace(1,60,100)

fig, ax = plt.subplots(1,4,figsize=(20,5),dpi =200, sharex = False)

cc = cycler(linestyle=['-', '--', '-.'])

eta_c = 0.9
eta_t = 0.9
# for a in ax:
#     a.set_prop_cycle(cc)

for TR in np.linspace(4,7,3):

    ax[0].plot(PR, ThermalEff(eta_c,eta_t,TR,PR), label = round(TR,2))
    ax[0].set_ylabel(r'$\eta_{thermal}$')
    
    ax[1].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR), label = round(TR,2))
    ax[1].set_ylabel(r'$\frac{\dot{W}_{net}}{\dot{m}c_pT_{t2}}$')

    ax[2].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR, turb_comp_works=True)[1], label = round(TR,2))
    ax[2].set_ylabel(r'$\frac{\dot{W}_{avail}}{\dot{m}c_pT_{t2}}$')
    ax[3].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR, turb_comp_works=True)[2], label = 'CompWork', color = 'k')
    #ax[3].plot(PR, TR-Work_per_flow(eta_c,eta_t,TR,PR, turb_comp_works=True)[3], label = r'$(T_{t4} - T_{t3})/T_{t2}$', color = 'r')
    ax[3].set_ylabel(r'$\frac{\dot{W}_{comp}}{\dot{m}c_pT_{t2}}$')
    
for a in ax:
    a.grid(alpha = 0.8)
    a.set_xlabel(r'Pressure ratio $\pi = \frac{P_{t3}}{P_{t2}}}$')
    lines = a.get_lines()
    l1=lines[-1]
    labelLine(l1,55,label=r'$TR=${}'.format(l1.get_label()),ha='center',va='center',align = True, fontsize = 10)
    labelLines(lines[:-1], xvals=(20, 60), align=True,fontsize = 10)
    
plt.tight_layout()

In [ ]:
TR = np.linspace(4,8,100)

fig, ax = plt.subplots(1,4,figsize=(10,3),dpi =150, sharex = False)

for PR in [10,20,40]:
    ax[0].plot(TR, ThermalEff(0.9,0.9,TR,PR), label = round(PR,2))
    ax[0].set_ylabel(r'$\eta_{thermal}$')
    
    ax[1].plot(TR, Work_per_flow(0.9,0.9,TR,PR), label = round(PR,2))
    ax[1].set_ylabel(r'$\frac{\dot{W}_{net}}{\dot{m}c_pT_{t2}}$')

    ax[2].plot(TR, Work_per_flow(0.9,0.9,TR,PR, turb_comp_works=True)[1], label = round(PR,2))
    ax[2].set_ylabel(r'$\frac{\dot{W}_{avail}}{\dot{m}c_pT_{t2}}$')
    ax[3].plot(TR, np.ones_like(TR)*Work_per_flow(0.9,0.9,TR,PR, turb_comp_works=True)[2], label = round(PR,2))
#     #ax[3].plot(PR, TR-Work_per_flow(0.9,0.9,TR,PR, turb_comp_works=True)[3], label = r'$(T_{t4} - T_{t3})/T_{t2}$', color = 'r')
    ax[3].set_ylabel(r'$\frac{\dot{W}_{comp}}{\dot{m}c_pT_{t2}}$')
    
for a in ax:
    a.grid(alpha = 0.8)
    a.set_xlabel(r'Temp ratio $\tau = \frac{T_{t4}}{T_{t2}}}$')
    lines = a.get_lines()
    l1=lines[-1]
    labelLine(l1,7,label=r'$PR=${}'.format(l1.get_label()),ha='center',va='center',align = True, fontsize = 8)
    labelLines(lines[:-1], xvals=(4, 6), align=True,fontsize = 8)
    
plt.tight_layout()

In [ ]:
PR = np.linspace(1,60,100)

fig, ax = plt.subplots(4,1,figsize=(8,10),dpi =150, sharex = False)

cc = cycler(linestyle=['-', '--', '-.'])

eta_c = 0.9
eta_t = 0.9
eta_pt = 0.95
# for a in ax:
#     a.set_prop_cycle(cc)

for TR in np.linspace(4,7,3):

    ax[0].plot(PR, ThermalEff(eta_c,eta_t,TR,PR), label = round(TR,2))
    ax[0].set_ylabel(r'$\eta_{thermal}$')
    
    ax[1].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR), label = round(TR,2))
    ax[1].set_ylabel(r'$\frac{\dot{W}_{net}}{\dot{m}c_pT_{t2}}$')

    ax[2].plot(PR, Work_per_flow_w_PT(eta_c,eta_t,eta_pt,TR,PR, details=False), label = round(TR,2))
    ax[2].set_ylabel(r'$\frac{\dot{W}_{PT}}{\dot{m}c_pT_{t2}}$')
    
    ax[3].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR, turb_comp_works=True)[2], label = 'CompWork', color = 'k')
    ax[3].set_ylabel(r'$\frac{\dot{W}_{comp}}{\dot{m}c_pT_{t2}}$')
    
for a in ax:
    a.grid(alpha = 0.8)
    a.set_xlabel(r'Pressure ratio $\pi = \frac{P_{t3}}{P_{t2}}}$')
    lines = a.get_lines()
    l1=lines[-1]
    labelLine(l1,55,label=r'$TR=${}'.format(l1.get_label()),ha='center',va='center',align = True, fontsize = 10)
    labelLines(lines[:-1], xvals=(20, 60), align=True,fontsize = 10)

# Vary compressor efficiency

In [ ]:
# Vary comp Eff
PR = np.linspace(1,60,100)

fig, ax = plt.subplots(4,1,figsize=(8,10),dpi =150, sharex = False)

cc = cycler(linestyle=['-', '--', '-.'])

eta_c = 0.9
eta_t = 0.9
eta_pt = 0.95
# for a in ax:
#     a.set_prop_cycle(cc)
TR = 5
for eta_c in np.linspace(0.7,0.95,5):

    ax[0].plot(PR, ThermalEff(eta_c,eta_t,TR,PR), label = round(eta_c,2))
    ax[0].set_ylabel(r'$\eta_{thermal}$')
    
    ax[1].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR, turb_comp_works=True)[2], label = round(eta_c,2) )
    ax[1].set_ylabel(r'$\frac{\dot{W}_{comp}}{\dot{m}c_pT_{t2}}$')
    
    ax[2].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR, turb_comp_works=True)[1], label = round(eta_c,2) )
    ax[2].set_ylabel(r'$\frac{\dot{W}_{Avail}}{\dot{m}c_pT_{t2}}$')
    
    ax[3].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR), label = round(eta_c,2) )
    ax[3].set_ylabel(r'$\frac{\dot{W}_{net}}{\dot{m}c_pT_{t2}}$')
    
for a in ax:
    a.grid(alpha = 0.8)
    a.set_xlabel(r'Pressure ratio $\pi = \frac{P_{t3}}{P_{t2}}}$')
    lines = a.get_lines()
    l1=lines[-1]
    labelLine(l1,55,label=r'$\eta_c=${}'.format(l1.get_label()),ha='center',va='center',align = True, fontsize = 9,
              bbox=dict(fc = "white", ec = "none",pad = 0))
    labelLines(lines[:-1], xvals=(10, 60), align=True,fontsize = 9,
               bbox=dict(fc = "white", ec = "none", pad = 0))

# Const. $\eta$ - FAR vs Thrust

An isentropic process has the following relationship between temperature and pressure:

$$ \frac{P_2}{P_1} = \left(\frac{T_2}{T_1}\right)^{\frac{\gamma}{\gamma-1}}$$

In [ ]:
def PR(TR, gamma = 1.4):
    k = (gamma-1)/gamma
#     print(k)
    return TR**(1/k)

def TR(PR, gamma =1.4):
    k = (gamma-1)/gamma
    return PR**k


## Combustor energy balance:

$$(\dot{m}_a + \dot{m}_f)\times c_p(T_{t4} - T_{t3}) = \dot{m}_f\times LHV$$

$$\therefore T_{t4} = T_{t3} + \frac{FAR\times LHV}{(1+FAR)c_p}$$

## Shaft power balance:

$$(\dot{m}_a +\dot{m}_f ) \times c_{p,t}(T_{t4} - T_{t5}) = \dot{m}_a\times c_{p,c}(T_{t3} - T_{t2})$$

$$T_{t5} = T_{t4} - \frac{c_{p,t}}{c_{p,c}}\frac{\dot{m}_a}{\dot{m}_f+\dot{m_a}}T_{t2}\left(\frac{T_{t3}}{T_{t_2}}-1\right) $$

$$T_{t5} = T_{t4} - T_{t2}\frac{c_{p,t}}{c_{p,c}}\frac{\left(\pi^{\gamma-1/\gamma}-1\right)}{FAR+1} $$

## Jet Velocity

$$T_{t5} = T_5 +\frac{V_j^2}{2c_p}$$

$$V_j = \sqrt{2c_p\left(T_{t5} - T_5\right)} = \sqrt{2c_pT_{t5}\left(1 - \frac{P_{amb}}{P_{t5}}^{\frac{\gamma -1}{\gamma}}\right)} $$

In [ ]:
def comb_balance(Tt3,FAR, LHV=43e6, cp = 1004):
    Tt4 = Tt3 + (FAR*LHV)/(1+FAR)/cp
    #print((FAR*LHV)/cp)
    return Tt4

def P3T3(press_ratio, Tt2= 298.15, Pt2 = 101.325e3, gamma = 1.4,):
    Pt3 = Pt2*press_ratio
    Tt3 = Tt2*TR(press_ratio)
    return Pt3, Tt3

def P5T5(Tt4, FAR, press_ratio, Tt2 = 298.15, Pt2 = 101.325e3, cp_t = 1244, cp_c =1004):
    Tt5 = Tt4 - 1/(FAR+1)*Tt2*(cp_t/cp_c)*(TR(press_ratio) - 1 )
    Pt4 = Pt2*press_ratio
    Pt5 = Pt4*PR(Tt5/Tt4)
    return Pt5, Tt5

def vj(Tt5,Pt5, Pamb = 101.325e3, gamma = 1.4, cp =1004):
    vj = np.sqrt(2*cp*Tt5*(1 - TR(Pamb/Pt5, 1.3)))   
    return vj

In [ ]:
def Turbojet(FAR, press_ratio, Fn=None, Tt2= 298.15, Pt2 = 101.325e3, print_steps = False, return_all = False):
    #Given the pressure ratio of the compressor
    # and Pt2, Tt2 we first calculate Pt3, Tt3:
    Pt3, Tt3 = P3T3(press_ratio, Tt2, Pt2)    
    #Combustor energy balance
    Tt4 = comb_balance(Tt3, FAR)  
    Pt4 = Pt3
    Pt5, Tt5 = P5T5(Tt4, FAR, press_ratio)
    F_spec = vj(Tt5,Pt5)
    if Fn is not None:
        mdot_a = Fn/F_spec
        mdot_f = FAR*mdot_a
    
    if print_steps:
        print("Pt3, Tt3 = {0:.2f} kPa, {1:.2f} K".format( Pt3/1000, Tt3))
        print("pi_c, tau_c = {0:.2f}, {1:.2f}".format(Pt3/Pt2, Tt3/Tt2))    
        print("Comp.SpecWork = {0:.3f}\n".format(Tt3-Tt2))
        print("Tt4 = {:.2f} K".format(Tt4))
        print("Tt4/Tt2 = {0:.2f}\n".format(Tt4/Tt2))
        print("Turb.SpecWork = = {0:.3f}\n".format(Tt4-Tt5))
        print("Pt5, Tt5 = = {0:.2f} kPa, {1:.2f} K".format(Pt5/1000, Tt5))
        print("Spec.Thrust = {0:.2f} m/s".format(F_spec))
        
    if Fn is not None and return_all:
        return F_spec, mdot_a, mdot_f, Tt2, Tt3, Tt4, Tt5, Pt2, Pt3, Pt4, Pt5
    elif Fn is not None:
        return F_spec, mdot_a, mdot_f  
    elif Fn is None and return_all:
        return F_spec, Tt2, Tt3, Tt4, Tt5, Pt2, Pt3, Pt4, Pt5
    elif not return_all:
        return F_spec
    

In [ ]:
Turbojet(0.05, 30, return_all=True)

In [ ]:
FAR = np.linspace(0.005,0.02,100)


fig, ax = plt.subplots(2,1,figsize=(7,7),dpi =150, sharex = False)

cc = cycler(linestyle=['-', '--', '-.'])

for p in np.linspace(20,50,3):
    F_spec, Tt2, Tt3, Tt4, Tt5, Pt2, Pt3, Pt4, Pt5 = Turbojet(FAR,p, return_all=True)
    ax[0].plot(FAR, F_spec, label = round(p,2))
    ax[0].set_ylabel(r'$\frac{F_N}{\dot{m}}$ [m/s]')
    ax[1].plot(FAR, Tt4/Tt2, label = round(p,2))
    ax[1].set_ylabel(r'$\dot{m}_a$ [m/s]')
    
for a in ax:
    a.grid(alpha = 0.8)
    a.set_xlabel(r'FAR')
    a.legend(title = "Pressure Ratio")
#     lines = a.get_lines()
#     l1=lines[-1]
#     labelLine(l1,0.006,label=r'$\pi_c=${}'.format(l1.get_label()),ha='center',va='center',align = True, fontsize = 8)
#     labelLines(lines[:-1], xvals=(0.006, 0.01), align=False,fontsize = 8)

In [ ]:

TurbEff_slider = widgets.FloatSlider(
    value=0.95,
    min=0.3,
    max=1.0,
    step=0.05,
    description=r"$\eta_{comp/turb}$:",
    readout_format='.2f',
    fontsize = 30,
)
def f(x):
    # Vary comp Eff
    PR = np.linspace(1,60,100)

    fig, ax = plt.subplots(2,1,figsize=(8,6),dpi =150, sharex = False)

    cc = cycler(linestyle=['-', '--', '-.'])

    eta_c = x
    eta_t = x
    eta_pt = 0.95
    # for a in ax:
    #     a.set_prop_cycle(cc)
    TR = 5
#     for eta_c in np.linspace(0.7,0.95,5):

    ax[0].plot(PR, ThermalEff(eta_c,eta_t,TR,PR), label = round(eta_c,2))
    ax[0].set_ylabel(r'$\eta_{thermal}$')

    ax[1].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR), label = round(eta_c,2) )
    ax[1].set_ylabel(r'$\frac{\dot{W}_{net}}{\dot{m}c_pT_{t2}}$')
    ax[1].set_xlabel(r'Pressure ratio $\pi = \frac{P_{t3}}{P_{t2}}}$')
    for a in ax:
        a.grid(alpha = 0.8)
        
#         lines = a.get_lines()
#         l1=lines[-1]
#         labelLine(l1,55,label=r'$\eta_c=${}'.format(l1.get_label()),ha='center',va='center',align = True, fontsize = 9,
#                   bbox=dict(fc = "white", ec = "none",pad = 0))
#         labelLines(lines[:-1], xvals=(10, 60), align=True,fontsize = 9,
#                    bbox=dict(fc = "white", ec = "none", pad = 0))

    title = r"Comp/Turb. $\eta$ = "+str(x)
    fig.suptitle(title)

In [ ]:
interact(f, x = TurbEff_slider)